In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import matplotlib.pyplot as plt
import gensim
import collections
import numpy as np
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
from spacy.lang.es import Spanish
from spacy.lang.es.stop_words import STOP_WORDS
import spacy
import pyLDAvis.gensim
import pyLDAvis

In [2]:
df_articles = pd.read_csv('../Data/Data_clean_csv/clean_dataframe.csv')
df_articles.dropna(inplace = True)

In [3]:
stopwords1 = set(stopwords.words('spanish'))
stopwords2 = set(STOP_WORDS)
stopwords = stopwords1.union(stopwords2)
nlp = spacy.load('es_core_news_md')

In [4]:
def cleaner(word):
    word = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', '', word, flags=re.MULTILINE)
    word = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', "", word)
    word = re.sub(r'ee.uu', 'eeuu', word)
    word = re.sub(r'\#\.', '', word)
    word = re.sub(r'\n', '', word)
    word = re.sub(r',', '', word)
    word = re.sub(r'\-', ' ', word)
    word = re.sub(r'\.{3}', ' ', word)
    word = re.sub(r'a{2,}', 'a', word)
    word = re.sub(r'é{2,}', 'é', word)
    word = re.sub(r'i{2,}', 'i', word)
    word = re.sub(r'ja{2,}', 'ja', word) 
    word = re.sub(r'á', 'a', word)
    word = re.sub(r'é', 'e', word)
    word = re.sub(r'í', 'i', word)
    word = re.sub(r'ó', 'o', word)
    word = re.sub(r'ú', 'u', word)  
    word = re.sub('[^a-zA-Z]', ' ', word)
    list_word_clean = []
    for w1 in word.split(" "):
        if  w1.lower() not in stopwords:
            list_word_clean.append(w1.lower())

    bigram_list = bigram[list_word_clean]
    out_text = lemmatization(" ".join(bigram_list))
    return out_text

In [5]:
def lemmatization(texts, allowed_postags=['NOUN']):
    texts_out = [ token.text for token in nlp(texts) if token.pos_ in 
                 allowed_postags and len(token.text)>2]
    return texts_out

In [6]:
def text_author (df):
    texts = {}
    for author in df['author'].unique():
        texts[author] = (' ').join(list(df[df['author']==author]['body']))
    return texts

In [7]:
def display_topics(model):
    tags = []
    for topic in model.print_topics():
        themes = re.findall( r'\"(.[^"]+).?', topic[1])
        tags.append(themes)
    tags = [tag for tag_list in tags for tag in tag_list]
    repeat = [item for item, count in collections.Counter(tags).items() if count > 2]
    
    for topic_idx, topic in enumerate(model.print_topics()):
        print ("Topic %d:" % (topic_idx+1))
        tags = re.findall( r'\"(.[^"]+).?', topic[1])
        tags = [tag for tag in tags if tag not in repeat]
        print (' '.join(tags), '\n')

In [8]:
def corpus_author (author):
    global bigram
    df_auth = df_articles[df_articles['author']==author]
    bigram = gensim.models.Phrases(df_auth['body'].astype('string').to_list()) 
    df_auth['body'] = df_auth['body'].apply(cleaner)
    dictionary = Dictionary(df_auth['body'].to_list())
    dictionary.compactify()
    dictionary.filter_extremes(no_below=2, no_above=0.97, keep_n=None)
    dictionary.compactify()
    corpus = [dictionary.doc2bow(text) for text in df_auth['body'].to_list()]
    return corpus, dictionary

In [9]:
corpus, dictionary = corpus_author('Valeria Moy')

<ipython-input-8-074e80c8aa92>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_auth['body'] = df_auth['body'].apply(cleaner)


In [11]:
ldamodel = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary, random_state=2)
display_topics(ldamodel)

Topic 1-Microeconomia
cambio precio personas 

Topic 2 - Inequidad de genero
pobreza mujeres desarrollo ingreso desigualdad 

Topic 3 - Política económica
gasto productividad nivel presupuesto 

Topic 4 - Economía empresarial
gasto economia empresas personas tema 

Topic 5 - Mujeres en la economía
desarrollo paises mujeres empresas


In [12]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.007535 -0.024960       1        1  25.726392
4     -0.018990 -0.018748       2        1  21.740493
0      0.020652  0.014331       3        1  20.403493
2      0.021225  0.008322       4        1  17.653491
1     -0.030422  0.021055       5        1  14.476130, topic_info=            Term        Freq       Total Category  logprob  loglift
44       pobreza  183.000000  183.000000  Default  30.0000  30.0000
495      mujeres  125.000000  125.000000  Default  29.0000  29.0000
482       ciudad   69.000000   69.000000  Default  28.0000  28.0000
262      mercado  143.000000  143.000000  Default  27.0000  27.0000
17   desigualdad   68.000000   68.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
276       sector   17.509955  145.083224   Topic5  -5.3410  -0.1819
295     empresas   16.931422  129.563259   Topic5  -5.3746  -0.1023
156     personas   17.085028  162.998982   Topic5  -5.3656  -0.3229
531     terminos   15.643797   93.759271   Topic5  -5.4537   0.1420
144    inversion   15.683919  127.502545   Topic5  -5.4512  -0.1628

[403 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
439       1  0.216578      abril
439       2  0.180482      abril
439       3  0.108289      abril
439       4  0.397059      abril
439       5  0.108289      abril
...     ...       ...        ...
716       1  0.160572  violencia
716       2  0.321145  violencia
716       3  0.120429  violencia
716       4  0.080286  violencia
716       5  0.321145  violencia

[1073 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 1, 3, 2])

In [13]:
corpus, dictionary = corpus_author('Denisse Dresser')

<ipython-input-8-074e80c8aa92>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_auth['body'] = df_auth['body'].apply(cleaner)


In [28]:
ldamodel = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary, random_state=8)
display_topics(ldamodel)


Topic 1 - Política
Tags:
democracia miembros partido mensaje 

Topic 2 - AMLO
Tags:
apoyo cambios amlo sistema 

Topic 3 - Violencia
Tags:
casos muertes violencia crisis miembros 

Topic 4 - Violencia de genero
Tags:
mujeres violencia sociedad hombres datos pruebas 

Topic 5 - Elecciones
Tags:
dinero reglas campa democracia voto pueblo 



In [29]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.017211  0.003697       1        1  26.624668
0     -0.009355 -0.001768       2        1  24.444497
4      0.007190 -0.037031       3        1  20.454500
2     -0.018274  0.018029       4        1  14.645752
3      0.037650  0.017073       5        1  13.830583, topic_info=           Term        Freq       Total Category  logprob  loglift
415       casos   53.000000   53.000000  Default  30.0000  30.0000
41          ley  121.000000  121.000000  Default  29.0000  29.0000
58   presidente  343.000000  343.000000  Default  28.0000  28.0000
125     muertes   34.000000   34.000000  Default  27.0000  27.0000
528      dinero   82.000000   82.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
239   impunidad    9.062499   69.182302   Topic5  -5.6493  -0.0543
547    recursos   11.092840  118.449159   Topic5  -5.4472  -0.3899
22   democracia    9.734856   89.392937   Topic5  -5.5778  -0.2390
48     millones    9.819945  110.719630   Topic5  -5.5691  -0.4443
229    derechos    7.938390   57.991666   Topic5  -5.7818  -0.0103

[421 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
518       1  0.229486  acciones
518       2  0.360622  acciones
518       3  0.196703  acciones
518       4  0.163919  acciones
518       5  0.065568  acciones
...     ...       ...       ...
979       1  0.194873     votos
979       2  0.194873     votos
979       3  0.389745     votos
979       4  0.064958     votos
979       5  0.129915     votos

[1099 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 5, 3, 4])

In [39]:
corpus, dictionary = corpus_author('Enrique Krauze')

<ipython-input-8-074e80c8aa92>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_auth['body'] = df_auth['body'].apply(cleaner)


In [41]:
ldamodel = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary)
display_topics(ldamodel)

Topic 1:
pueblo ideas tiempos millones 

Topic 2:
elecciones medios movimiento 

Topic 3:
guerra mundo personas 

Topic 4:
partido partidos pan cambio 

Topic 5:
cultura sociedad tiempos pueblo 



In [42]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.022642 -0.004241       1        1  37.934486
2     -0.006229 -0.004334       2        1  23.297421
0     -0.016169  0.008017       3        1  15.818265
3      0.006132  0.016167       4        1  12.482829
4     -0.006376 -0.015609       5        1  10.466998, topic_info=           Term        Freq       Total Category  logprob  loglift
249     partido  216.000000  216.000000  Default  30.0000  30.0000
144        vida  425.000000  425.000000  Default  29.0000  29.0000
396     reforma  572.000000  572.000000  Default  28.0000  28.0000
102    historia  477.000000  477.000000  Default  27.0000  27.0000
215     cultura  253.000000  253.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
936   violencia   21.803266  232.273556   Topic5  -5.7629  -0.1089
235      guerra   24.656497  327.722847   Topic5  -5.6399  -0.3302
382  movimiento   20.461926  212.848858   Topic5  -5.8263  -0.0851
392     proceso   18.739767  182.035985   Topic5  -5.9143  -0.0166
214      crisis   18.048816  170.026837   Topic5  -5.9518   0.0141

[414 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3567      2  0.541420  acarreos
3881      2  0.289706   accesos
3881      3  0.579412   accesos
408       1  0.304579   actitud
408       2  0.278472   actitud
...     ...       ...       ...
597       3  0.297760    vuelta
597       4  0.119104    vuelta
597       5  0.079403    vuelta
2203      1  0.392230   vulgata
2203      4  0.392230   vulgata

[999 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 4, 5])

In [43]:
corpus, dictionary = corpus_author('John Ackerman')

<ipython-input-8-074e80c8aa92>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_auth['body'] = df_auth['body'].apply(cleaner)


In [48]:
ldamodel = LdaModel(corpus=corpus, num_topics=4, id2word=dictionary)
display_topics(ldamodel)

Topic 1:
partidos ciudadanos caso proceso sistema autoridades 

Topic 2:
mujeres crisis pueblo violencia sistema ley 

Topic 3:
empresas proceso mundo impunidad democracia 

Topic 4:
medios pueblo gobierno pesos democracia 



In [49]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.004758 -0.005679       1        1  37.680643
1     -0.040364  0.007278       2        1  27.286877
0      0.027535  0.029643       3        1  17.956790
2      0.017587 -0.031243       4        1  17.075690, topic_info=           Term        Freq       Total Category  logprob  loglift
706     mujeres   39.000000   39.000000  Default  30.0000  30.0000
297     partido   73.000000   73.000000  Default  29.0000  29.0000
150    partidos   37.000000   37.000000  Default  28.0000  28.0000
100    millones  109.000000  109.000000  Default  27.0000  27.0000
606  aeropuerto   11.000000   11.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
225       redes    5.401839   28.816644   Topic4  -5.6617   0.0933
152       pesos    6.054036   46.039370   Topic4  -5.5477  -0.2613
69     sociedad    5.659866   37.234430   Topic4  -5.6150  -0.1163
158      pueblo    5.585986   67.724026   Topic4  -5.6281  -0.7277
68      sistema    5.268418   44.022958   Topic4  -5.6867  -0.3555

[304 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
116       1  0.477574   acciones
116       2  0.212255   acciones
116       3  0.159191   acciones
116       4  0.106127   acciones
1023      2  0.582259  activista
...     ...       ...        ...
168       4  0.156602       voto
557       1  0.418104      votos
557       2  0.261315      votos
557       3  0.235183      votos
557       4  0.104526      votos

[648 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 3])

In [50]:
corpus, dictionary = corpus_author('Ricardo Raphael')

<ipython-input-8-074e80c8aa92>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_auth['body'] = df_auth['body'].apply(cleaner)


In [55]:
ldamodel = LdaModel(corpus=corpus, num_topics=4, id2word=dictionary)
display_topics(ldamodel)

Topic 1:
ley grupo autoridad mujeres organizaciones secretario 

Topic 2:
coronavirus millones meses vida medicamentos paises pacto 

Topic 3:
caso proceso gobernador secretario tribunales ley 

Topic 4:
millones pesos partido numero meses proposito vida 



In [56]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.043128 -0.005590       1        1  34.484217
2     -0.021208 -0.039017       2        1  23.547288
1      0.012868  0.020258       3        1  22.658637
0     -0.034788  0.024349       4        1  19.309858, topic_info=                Term       Freq      Total Category  logprob  loglift
43          millones  60.000000  60.000000  Default  30.0000  30.0000
35               ley  21.000000  21.000000  Default  29.0000  29.0000
870  transcripciones   6.000000   6.000000  Default  28.0000  28.0000
271            grupo  18.000000  18.000000  Default  27.0000  27.0000
762     medicamentos  12.000000  12.000000  Default  26.0000  26.0000
..               ...        ...        ...      ...      ...      ...
248       septiembre   4.048099  13.452330   Topic4  -5.6365   0.4436
155          persona   4.349837  16.907247   Topic4  -5.5646   0.2870
580         tragedia   4.272138  17.796949   Topic4  -5.5826   0.2176
219             caso   4.416511  33.394043   Topic4  -5.5494  -0.3785
297             vida   4.248712  26.479666   Topic4  -5.5881  -0.1852

[284 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
368       1  0.546129    actos
368       2  0.234055    actos
368       3  0.156037    actos
368       4  0.156037    actos
216       1  0.220256  agentes
...     ...       ...      ...
254       2  0.343156      voz
254       3  0.343156      voz
254       4  0.137262      voz
660       1  0.387059   yerros
660       2  0.387059   yerros

[642 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1])